In [1]:
import os

%load_ext dotenv
%dotenv

In [2]:
import boto3
from boto3 import Session
from sagemaker import clarify

role = os.environ["ROLE"]
sagemaker_session = Session()
sagemaker_client = boto3.client("sagemaker")
iam_client = boto3.client("iam")
default_bucket = os.environ["BUCKET"]
default_prefix = "sagemaker/DEMO-sagemaker-clarify"

clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role, instance_count=1, instance_type="ml.m5.xlarge", sagemaker_session=sagemaker_session
)

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\kamil\AppData\Local\sagemaker\sagemaker\config.yaml


KeyError: 'ROLE'